In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

def logsig(x):
    return 1/(1+np.exp(-x))

In [49]:
def  soundingdata(a):
    #时间
    data1=pd.read_csv("D:/20年海洋/数据/探空数据/12个月南海探空气象数据/"+str(a)+'/'+str(a)+"_17010108.csv",encoding='gbk')
    data2=pd.read_excel("D:/20年海洋/数据/探空数据/12个月全部站点信息/12个月去除后南海区域探空站点.xlsx")
    data3=data2.loc[data2['站点']==a]
    x_data=np.array(data1['高度'])-np.array(data3['海拔高度'])
    lon=np.array(data3['经度'])
    lat=np.array(data3['维度'])
    for j in range(len(x_data)):
        if x_data[j]>3000:
            print(j)
            break
    x_data=x_data[0:j]
    y1_data=np.array(data1['水汽压'])[0:j]
    y2_data=np.array(data1['温度'])[0:j]+273.15
    y3_data=np.array(data1['气压'])[0:j]
    y4_data=np.array(data1['水汽混合比'])[0:j]
    return x_data,y1_data,y2_data,y3_data,y4_data,j,lon,lat

In [50]:
def  simulateddata(a):
    #时间
    data1=pd.read_excel("D:/20年海洋/3000m以下/数据预处理/双线性插值/20170101_00.xlsx",sheet_name=str(a))
    height=np.array(data1['z'])
    monie=np.array(data1['e'])
    moniTem=np.array(data1['Tem'])
    monipressure=np.array(data1['pressure'])
    moniQVAPOR=np.array(data1['QVAPOR'])
    return height,monie,moniTem,monipressure,moniQVAPOR

In [51]:
def neuralnetwork(x_data,y1_data,y2_data,y3_data,y4_data,j,height):
    # normalize the original data
    samplein = np.mat([x_data]) #1*i
    sampleinminmax = np.array([samplein.min(axis=1).T.tolist()[0],samplein.max(axis=1).T.tolist()[0]]).transpose()#1*2
    sampleout = np.mat([y1_data,y2_data,y3_data,y4_data])#4*i
    sampleoutminmax = np.array([sampleout.min(axis=1).T.tolist()[0],sampleout.max(axis=1).T.tolist()[0]]).transpose()#4*2
    sampleinnorm = (2*(np.array(samplein.T).astype(float)-sampleinminmax.transpose()[0])/(sampleinminmax.transpose()[1]-sampleinminmax.transpose()[0])-1).transpose()
    sampleoutnorm = (2*(np.array(sampleout.T).astype(float)-sampleoutminmax.transpose()[0])/(sampleoutminmax.transpose()[1]-sampleoutminmax.transpose()[0])-1).transpose()
    testin=np.mat([height])#1*45
    testinnorm = (2*(np.array(testin.T).astype(float)-sampleinminmax.transpose()[0])/(sampleinminmax.transpose()[1]-sampleinminmax.transpose()[0])-1).transpose()
    #initial the parameters
    maxepochs =100000
    learnrate = 0.001
    errorfinal = 0.5*10**(-8)
    samnum = j
    indim = 1
    outdim = 4
    hiddenunitnum = 10
    #run
    w1 = 2*np.random.rand(hiddenunitnum,indim)-1
    b1 = 2*np.random.rand(hiddenunitnum,1)-1
    w2 = 2*np.random.rand(outdim,hiddenunitnum)-1
    b2 = 2*np.random.rand(outdim,1)-1
    errhistory = []
    for i in range(maxepochs):
        hiddenout = logsig((np.dot(w1,sampleinnorm).transpose()+b1.transpose())).transpose()
        networkout = (np.dot(w2,hiddenout).transpose()+b2.transpose()).transpose()
        err = sampleoutnorm - networkout
        sse = sum(sum(err**2))
    #Use the err of the whole dataset as the err, rather than one subject, aiming of reduce the err fast
        errhistory.append(sse)
        if sse < errorfinal:
            break
        delta2 = err
        delta1 = np.dot(w2.transpose(),delta2)*hiddenout*(1-hiddenout)
        dw2 = np.dot(delta2,hiddenout.transpose())
        db2 = np.dot(delta2,np.ones((samnum,1)))
        dw1 = np.dot(delta1,sampleinnorm.transpose())
        db1 = np.dot(delta1,np.ones((samnum,1)))
        w2 += learnrate*dw2
        b2 += learnrate*db2
        w1 += learnrate*dw1
        b1 += learnrate*db1
    #test
    hiddenout1 = logsig((np.dot(w1,testinnorm).transpose()+b1.transpose())).transpose()
    networkout1 = (np.dot(w2,hiddenout1).transpose()+b2.transpose()).transpose()
    diff = sampleoutminmax[:,1]-sampleoutminmax[:,0]
    networkout21 = (networkout1+1)/2
    networkout21[0] = networkout21[0]*diff[0]+sampleoutminmax[0][0]
    networkout21[1] = networkout21[1]*diff[1]+sampleoutminmax[1][0]
    networkout21[2] = networkout21[2]*diff[2]+sampleoutminmax[2][0]
    networkout21[3] = networkout21[3]*diff[3]+sampleoutminmax[3][0]
    return  networkout21[0],networkout21[1],networkout21[2],networkout21[3]

In [53]:
#时间
data=pd.read_csv("D:/20年海洋/数据/探空数据/分月站点信息/1/2017-01-01 08.csv",encoding='utf-8')
data1=data.loc[data['层数']>20]
zhanhao=np.array(data1.iloc[:,0])
length=len(zhanhao)
for i in range(1):
    a=zhanhao[i]
    zh=np.array([a]*45)
    x_data,y1_data,y2_data,y3_data,y4_data,j,lon,lat=soundingdata(a)
    long=np.array([lon]*45).flatten()
    lati=np.array([lat]*45).flatten()
    height,monie,moniTem,monipressure,moniQVAPOR=simulateddata(a)
    alignede,alignedTem,alignedpressure,alignedQVAPOR=neuralnetwork(x_data,y1_data,y2_data,y3_data,y4_data,j,height)
    
    zh = pd.Series(zh)
    zh.name = 'number'
    long = pd.Series(long)
    long.name = 'lon'
    lati = pd.Series(lati)
    lati.name = 'lat'
    height = pd.Series(height)
    height.name = 'z'
    monie = pd.Series(monie)
    monie.name = 'monie'
    alignede = pd.Series(alignede)
    alignede.name = 'alignede'
    moniTem = pd.Series(moniTem)
    moniTem.name = 'moniTem'
    alignedTem = pd.Series(alignedTem)
    alignedTem.name = 'alignedTem'
    monipressure = pd.Series(monipressure)
    monipressure.name = 'monipressure'
    alignedpressure = pd.Series(alignedpressure)
    alignedpressure.name = 'alignedpressure'
    moniQVAPOR = pd.Series(moniQVAPOR)
    moniQVAPOR.name = 'moniQVAPOR'
    alignedQVAPOR = pd.Series(alignedQVAPOR)
    alignedQVAPOR.name = 'alignedQVAPOR'
    
    data2=pd.concat([zh,long,lati,height,monie,alignede,moniTem,alignedTem,monipressure,alignedpressure,moniQVAPOR,alignedQVAPOR],axis=1)
    #时间
    data2.to_csv("D:/20年海洋/3000m以下/数据预处理/神经网络/20170101_00.csv" , encoding = "utf-8", header=True, index=None)

18


In [54]:
for i in range(1,length):
    a=zhanhao[i]
    zh=np.array([a]*45)
    x_data,y1_data,y2_data,y3_data,y4_data,j,lon,lat=soundingdata(a)
    long=np.array([lon]*45).flatten()
    lati=np.array([lat]*45).flatten()
    height,monie,moniTem,monipressure,moniQVAPOR=simulateddata(a)
    alignede,alignedTem,alignedpressure,alignedQVAPOR=neuralnetwork(x_data,y1_data,y2_data,y3_data,y4_data,j,height)
    
    zh = pd.Series(zh)
    zh.name = 'number'
    long = pd.Series(long)
    long.name = 'lon'
    lati = pd.Series(lati)
    lati.name = 'lat'
    height = pd.Series(height)
    height.name = 'z'
    monie = pd.Series(monie)
    monie.name = 'monie'
    alignede = pd.Series(alignede)
    alignede.name = 'alignede'
    moniTem = pd.Series(moniTem)
    moniTem.name = 'moniTem'
    alignedTem = pd.Series(alignedTem)
    alignedTem.name = 'alignedTem'
    monipressure = pd.Series(monipressure)
    monipressure.name = 'monipressure'
    alignedpressure = pd.Series(alignedpressure)
    alignedpressure.name = 'alignedpressure'
    moniQVAPOR = pd.Series(moniQVAPOR)
    moniQVAPOR.name = 'moniQVAPOR'
    alignedQVAPOR = pd.Series(alignedQVAPOR)
    alignedQVAPOR.name = 'alignedQVAPOR'
    
    data2=pd.concat([zh,long,lati,height,monie,alignede,moniTem,alignedTem,monipressure,alignedpressure,moniQVAPOR,alignedQVAPOR],axis=1)
    #时间
    data2.to_csv("D:/20年海洋/3000m以下/数据预处理/神经网络/20170101_00.csv" ,mode='a', encoding = "utf-8", header=None, index=None)

22
14
17
12
12
16
14
15
10
18
17
20
21
12
13
3
12
